In [1]:
import os

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from vllm import LLM, SamplingParams

# Custom libraries
from vqa_dataset import PromptDataset,prompt_collate,create_template

INFO 11-10 17:46:55 [__init__.py:216] Automatically detected platform cuda.


In [2]:
## User Input ##
model       = "llava-hf/llava-1.5-7b-hf"
task        = "classifcation"
save_every  = 50
options     = True
out_dit     = "out"
model_dir   = "/pasteur/u/rdcunha/models"

## Envs:
os.environ["HF_HOME"] = model_dir
os.environ["TRANSFORMERS_CACHE"] = model_dir
os.environ["HUGGINGFACE_HUB_CACHE"] = model_dir
os.environ["VLLM_CACHE_ROOT"]  = model_dir
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Image.MAX_IMAGE_PIXELS = None

In [24]:
llm = LLM(
    hf_overrides= {"cache_dir": model_dir},
    download_dir=model_dir,
    model=model,
    trust_remote_code=True,       # required for LLaVA
    tensor_parallel_size=1,       # >1 if you have multiple GPUs
    gpu_memory_utilization=0.95,  # pack VRAM aggressively
    max_model_len=4096,           # tune to your context needs
)

INFO 11-10 17:55:29 [utils.py:233] non-default args: {'trust_remote_code': True, 'download_dir': '/pasteur/u/rdcunha/models', 'max_model_len': 4096, 'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'hf_overrides': {'cache_dir': '/pasteur/u/rdcunha/models'}, 'model': 'llava-hf/llava-1.5-7b-hf'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 11-10 17:55:29 [model.py:547] Resolved architecture: LlavaForConditionalGeneration
INFO 11-10 17:55:29 [model.py:1510] Using max model len 4096
INFO 11-10 17:55:29 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.


/pasteur/u/rdcunha/inference/.venv/lib/python3.13/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO 11-10 17:55:39 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:41 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:41 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='llava-hf/llava-1.5-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-1.5-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir='/pasteur/u/rdcunha/models', load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser=''), observability_config=ObservabilityC

...

(EngineCore_DP0 pid=100109) INFO 11-10 17:55:46 [gpu_model_runner.py:2602] Starting to load model llava-hf/llava-1.5-7b-hf...
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:46 [gpu_model_runner.py:2634] Loading model from scratch...
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:46 [layer.py:444] MultiHeadAttention attn_backend: _Backend.FLASH_ATTN, use_upstream_fa: False
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:46 [cuda.py:366] Using Flash Attention backend on V1 engine.
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:46 [weight_utils.py:392] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:00,  2.71it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:00<00:00,  2.71it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  2.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:01<00:00,  2.53it/s]
(EngineCore_DP0 pid=100109) 


(EngineCore_DP0 pid=100109) INFO 11-10 17:55:48 [default_loader.py:267] Loading weights took 1.25 seconds
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:48 [gpu_model_runner.py:2653] Model loading took 13.1343 GiB and 1.827461 seconds
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:48 [gpu_model_runner.py:3344] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 14 image items of the maximum feature size.
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:54 [backends.py:548] Using cache directory: /pasteur/u/rdcunha/models/torch_compile_cache/e000171312/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:54 [backends.py:559] Dynamo bytecode transform time: 5.43 s
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:56 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.772 s
(EngineCore_DP0 pid=100109) INFO 11-10 17:55:57 [monitor.py:34] torch.compile takes 5.43 s in total
(EngineCore_DP0 pid=100

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 25.39it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:01<00:00, 33.71it/s]


(EngineCore_DP0 pid=100109) INFO 11-10 17:56:03 [gpu_model_runner.py:3480] Graph capturing finished in 4 secs, took 2.34 GiB
(EngineCore_DP0 pid=100109) INFO 11-10 17:56:03 [core.py:210] init engine (profile, create kv cache, warmup model) took 14.36 seconds
INFO 11-10 17:56:03 [llm.py:306] Supported_tasks: ['generate']


In [25]:
task     = "all_cls"
mbu_root = f"/pasteur/u/rdcunha/data_cache/mmbu/final_data/VLMEvalData_v2/LMUData/{task}"

data_root= os.path.join(mbu_root, 'all_cls_closed_subsampled.tsv')

data = pd.read_csv(data_root,sep='\t')
filtered_ = data[data["question_type"] == "expert"]
df_ = filtered_[~filtered_["dataset"].isin([ "isic2018",'herlev',"breakhis_400x","breakhis_200x"])]

In [26]:
# quality tests
#print(f"Datasets:{filtered_["dataset"].unique()}")
no_opitons_d = 0
for options in filtered_["options"].unique():
    #print(options)
    if len(options.split(","))  < 2: no_opitons_d +=1
    else:pass

print(no_opitons_d)

0


In [27]:
from torch.utils.data import Dataset, DataLoader
ds = PromptDataset(df=df_)
questions_data_loaders = DataLoader(ds, 
                                    batch_size=10, 
                                    shuffle=False,
                                    collate_fn=prompt_collate, 
                                    num_workers=10,
                                    persistent_workers=True, 
                                    pin_memory=True, 
                                    prefetch_factor=4)


# Test

In [28]:
sampling_params = SamplingParams(temperature=0, max_tokens=512)
for items in tqdm(questions_data_loaders, desc="Processing batches"):

    # Generate only for new items
    conversations = [create_template(it) for it in items]
    outputs = llm.generate(conversations, sampling_params=sampling_params)
    break

Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed

Processing batches:   0%|          | 0/735 [00:12<?, ?it/s]


# Run

In [34]:
import os
import json
from tqdm import tqdm
model_name = model
try:
    model_name = model_name.split('/')[1]
except:
    model_name = model_name
    
save_path = f"{model_name}_{task}_expert_closed.jsonl"
save_file = os.path.join(out_dit, save_path)

# --- Step 1: Collect already processed IDs ---
existing_ids = set()
if os.path.exists(save_file):
    with open(save_file, "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                existing_ids.add(data["index"])
            except json.JSONDecodeError:
                continue  # skip corrupted lines

print(f"Found {len(existing_ids)} already processed items. Skipping them...")

# --- Step 2: Run inference only for new IDs ---
saved_items = []
counter = 0
sampling_params = SamplingParams(temperature=0, max_tokens=512)

with open(save_file, "a") as f:
    for items in tqdm(questions_data_loaders, desc="Processing batches"):
        # Filter out items whose IDs already exist
        new_items = [it for it in items if it["index"] not in existing_ids]
        if not new_items:
            continue  # nothing new in this batch

        ### THIS USUALLY NEEDS TO BE EDITED ###
        messages = [create_template(item) for item in items]
        inputs = processor.apply_chat_template( messages, add_generation_prompt=True, padding=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=torch.bfloat16)
        input_len = inputs["input_ids"].shape[-1]
        try:
            with torch.inference_mode():
                generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
                outputs = []
                for g in generation:
                    g = g[input_len:]
                    decoded = processor.decode(g, skip_special_tokens=True)
                    outputs.append(decoded)
        except: 
            print(f"could not generate for {items}")
            continue
        
            

            #####
       
        #except:
        #    print(f"could not generate for {items}")
        #    continue
       ### THIS USUALLY NEEDS TO BE EDITED ###

        for it, output in zip(new_items, outputs):
           
    
            for it, output in zip(new_items, outputs):
                answer = output.outputs[0].text
                saved_items.append({
                    "index": it["index"],
                    "question": it["question"],
                    "options": it["options"],
                    "image_path": it["image_path"],
                    "image_scale": it["image_scale"],
                    "scaled_width": it["scaled_width"],
                    "scaled_height": it["scaled_height"],
                    "dataset": it["dataset"],
                    "answer": answer
                })
            existing_ids.add(it["index"])  # add to skip list in case of crash recovery
            counter += 1

            # Save every N examples
            if counter % save_every == 0:
                print(f"Saving at {save_file}")
                for s in saved_items:
                    f.write(json.dumps(s) + "\n")
                f.flush()
                saved_items = []

        #print("Could not run batch:",items)
    # Save remaining items
    if saved_items:
        for s in saved_items:
            f.write(json.dumps(s) + "\n")


Found 0 already processed items. Skipping them...


Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]


NameError: name 'processor' is not defined

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [ ]:
import os
import json
from tqdm import tqdm

model_name = model
try:
    model_name = model_name.split('/')[1]
except:
    model_name = model_name
    
save_path = f"{model_name}_{task}_expert_closed.jsonl"
save_file = os.path.join(out_dit, save_path)

# --- Step 1: Collect already processed IDs ---
existing_ids = set()
if os.path.exists(save_file):
    with open(save_file, "r") as f:
        for line in f:
            try:
                data = json.loads(line)
                existing_ids.add(data["index"])
            except json.JSONDecodeError:
                continue  # skip corrupted lines

print(f"Found {len(existing_ids)} already processed items. Skipping them...")

# --- Step 2: Run inference only for new IDs ---
saved_items = []
counter = 0
sampling_params = SamplingParams(temperature=0, max_tokens=512)

with open(save_file, "a") as f:
    for items in tqdm(questions_data_loaders, desc="Processing batches"):
        # Filter out items whose IDs already exist
        new_items = [it for it in items if it["index"] not in existing_ids]
        if not new_items:
            continue  # nothing new in this batch

        ### THIS USUALLY NEEDS TO BE EDITED ###
     
        try:
            conversations = [create_template(it) for it in new_items]
            outputs = llm.generate(conversations, sampling_params=sampling_params)
           
        except: 
            print(f"could not generate for {items}")
            continue
        
            

            #####
       
        #except:
        #    print(f"could not generate for {items}")
        #    continue
       ### THIS USUALLY NEEDS TO BE EDITED ###

        for it, output in zip(new_items, outputs):
            answer = output.outputs[0].text
            saved_items.append({
                "index": it["index"],
                "question": it["question"],
                "options": it["options"],
                "image_path": it["image_path"],
                "image_scale": it["image_scale"],
                "scaled_width": it["scaled_width"],
                "scaled_height": it["scaled_height"],
                "dataset": it["dataset"],
                "class_label":it["class_label"],
                
                "answer": answer
            })
            existing_ids.add(it["index"])  # add to skip list in case of crash recovery
            counter += 1

            # Save every N examples
            if counter % save_every == 0:
                print(f"Saving at {save_file}")
                for s in saved_items:
                    f.write(json.dumps(s) + "\n")
                f.flush()
                saved_items = []

        #print("Could not run batch:",items)
    # Save remaining items
    if saved_items:
        for s in saved_items:
            f.write(json.dumps(s) + "\n")


Found 0 already processed items. Skipping them...


Processing batches:   0%|          | 0/735 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed

Processing batches:   0%|  | 1/735 [00:00<02:16,  5.38it/s]

...

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed

Processing batches:  35%|▎| 259/735 [03:01<05:21,  1.48it/s

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/10 [00:00<?, ?it/s, est. speed